In [4]:
using LinearAlgebra

In [9]:
ρ = [1 2 3; 4 5 6 ; 7 8 9]

3×3 Matrix{Int64}:
 1  2  3
 4  5  6
 7  8  9

In [29]:
size(ρ)

(3, 3)

In [10]:
r = vec(ρ)

9-element Vector{Int64}:
 1
 4
 7
 2
 5
 8
 3
 6
 9

In [11]:
reshape(r, (3,3))

3×3 Matrix{Int64}:
 1  2  3
 4  5  6
 7  8  9

In [24]:
function vectorize(x)
    return vec(x)
end

function unvectorize(x)
    # Assumes the input object to be a reshapable to a square matrix.
    d = trunc(Int, sqrt(length(x)))
    return reshape(x, (d,d))
end


unvectorize (generic function with 1 method)

In [25]:
ρ = [1 2 3; 4 5 6 ; 7 8 9]

3×3 Matrix{Int64}:
 1  2  3
 4  5  6
 7  8  9

In [26]:
vectorize(ρ)

9-element Vector{Int64}:
 1
 4
 7
 2
 5
 8
 3
 6
 9

In [27]:
unvectorize(vectorize(ρ))

3×3 Matrix{Int64}:
 1  2  3
 4  5  6
 7  8  9

In [30]:
function conjugate_matrix(x)
    return conj.(x)
end

conjugate_matrix (generic function with 1 method)

In [42]:
function lindblad_operator_to_vectorized_form(L)
    d = size(L)[1]
    ide = Matrix{Float64}(I, d, d)
    return kron(conjugate_matrix(L), L) - 0.5 * kron(transpose(L' * L), ide) - 0.5 * kron(ide, L' * L)
end

lindblad_operator_to_vectorized_form (generic function with 1 method)

In [44]:
function unitary_evolution_to_vectorized_form(H)
    d = size(H)[1]
    ide = Matrix{Float64}(I, d, d)
    return - 1im * (kron(ide, H) -  kron(transpose(H), ide))
end

unitary_evolution_to_vectorized_form (generic function with 1 method)

In [45]:
function liouvillian(H, C_l)
    L = unitary_evolution_to_vectorized_form(H)
    for C in C_l
        L += lindblad_operator_to_vectorized_form(C)
    end
    return L
end

liouvillian (generic function with 1 method)

In [46]:
function adjoint_liouvillian(H, C_l)
    d = size(H)[1]
    ide = Matrix{Float64}(I, d, d)

    hamiltonian_part = 1im * kron(ide, H) - 1im * kron(transpose(H), ide)
    L_dagger = hamiltonian_part

    for C in C_l
        L_dagger += kron(transpose(L), L') - 0.5 * kron(transpose(L' * L), ide) - 0.5 * kron(ide, L' * L)
    end

    return L_dagger
end

adjoint_liouvillian (generic function with 1 method)

In [7]:
function gillespie_partial_monitoring(
    H::Matrix{ComplexF64},
    M_l::Vector{Matrix{ComplexF64}},
    S_l::Vector{Matrix{ComplexF64}},
    ρ0::Matrix{ComplexF64},
    t_final::Float64,
    dt::Float64,
    number_trajectories::Int64,
    verbose::Bool=false)

    # Range of times.
    t_range = 0.:dt:t_final

    # M_l is the list of jumps corresponding to monitored channels.
    # S_l is the list of jumps corresponding to un-monitored channels.

    # Creates the appropriate identity matrix.
    d = size(H)[1]
    ide = Matrix{Float64}(I, d, d)

    # Creates the J operator.
    J = zero(ide)
    # Cycle over the monitored operators.
    for M in M_l
        J += M' * M
    end

    # Vectorized version of J.
    vect_J = vectorize(J)

    # Vectorized form of L_0.
    # Hamiltonian part.
    vect_L0 = -1im * kron(ide, H) + 1im * kron(transpose(H), ide)
    # Cycle over the un-monitored operators.
    for S in S_l
        vect_L0 += kron(conj.(S), S) - 0.5 * kron(ide, S' * S) - 0.5 * (transpose(S' * S), ide)
    end
    # Cycle over the monitored operators.
    for M in M_l
        vect_L0 += - 0.5 * kron(ide, M' * M) - 0.5 * kron(transpose(M' * M), ide)
    end
    
    # Vectorized form of L_0^\dagger.
    # Hamiltonian part.
    vect_L0_dagger = 1im * kron(ide, H) - 1im * kron(transpose(H), ide)
    # Cycle over the un-monitored operators.
    for S in S_l
        vect_L0_dagger += kron(transpose(S), S') - 0.5 * kron(transpose(S' * S), ide) - 0.5 * kron(ide, S' * S)
    end
    # Cycle over the monitored operators.
    for M in M_l
        vect_L0_dagger += - 0.5 * kron(transpose(M' * M), ide) - 0.5 * kron(ide, M' * M)
    end

    # Pre-computation stage.
    # Creates the list of the no-jump evolution operators and the non-state dependent part of the waiting time distribution.
    V = Matrix{ComplexF64}[] 
    Qs = Matrix{ComplexF64}[]
    for t in t_range
        ev_op = exp(vect_L0 * t)
        push!(V, ev_op)
        nsd_wtd = unvectorize(exp(vect_L0_dagger * t) * vect_J)
        push!(Qs, nsd_wtd)
    end

    # TODO: Some way of quantifying the error (like the norm of the latest Qs in the normal version)

    # Vector for the results of the computation.
    trajectories_results = Array{Dict{String, Any}}[]

    # Cycle over the trajectories.
    for trajectory in 1:number_trajectories
        # Initial state.
        ρ = ρ0
        # Absolute time.
        τ = 0

        # Creates the array of results for the single trajectory, and pushes the initial state as a fictitious fist jump.
        results = Dict{String, Any}[]
        dict_initial = Dict("AbsTime" => 0,
            "TimeSinceLast" => 0,
            "JumpChannel" => nothing,
            "ρAfter" => ρ0)
        push!(results, dict_initial)

        while τ < t_final 
            dict_jump = Dict()

            # Compute the waiting time distribution, exploiting the pre-computed part.
            Ps = Float64[]
            for Q in Qs
                wtd =  tr(Q * ρ)
                push!(Ps, wtd)
            end

            # Sample from the waiting time distribution.
            n_T = sample(1:length(t_range), Weights(Ps))

            # Increase the absolute time.
            τ += t_range[n_T]
            merge!(dict_jump, Dict("AbsTime" => τ, "TimeSinceLast" => t_range[n_T]))

            # Update the state.
            vect_ρ = V[n_T] * vectorize(ρ)
            ρ = unvectorize(vect_ρ)
            # Chooses where to jump.
            weights = Float64[]
            for M in M_l
                weight = real(tr(M' * M * ρ))
                push(weights, weight)
            end
            n_jump = sample(1:lenght(M_l), Weights(weights))
            merge!(dict_jump, Dict("JumpChannel" => n_jump))
            # Update the state after the jump.
            ρ = M_l[n_jump] * ρ * (M_l[n_jump])'
            norm_state = real(tr(ρ))
            # Renormalize the state.
            ρ = ρ / norm_state
            merge!(dict_jump, Dict("ρAfter" => ψ))

            if verbose
                println(string(dict_jump))
            end

            push!(results, dict_jump)
        end

        push!(trajectories_results, results)
    end

    return trajectories_results, V, t_range
end

gillespie_partial_monitoring (generic function with 2 methods)